# Learn v75 med walkthrough-metoden

In [1]:
import pandas as pd 
import numpy as np 
from catboost import CatBoostClassifier,Pool,cv,utils

import sys
sys.path.append('C:\\Users\peter\\Documents\\MyProjects\\PyProj\\Trav\\spel')
import V75_scraping as vs

In [2]:
# df skall innehålla datum,avd,vodds
def proba_order_score(df_, y,proba):
    kassa=1000
    df = df_.copy()
    df['proba'] = proba[:,1]
    df['f'] = (df.proba*df.vodds - 1) / (df.vodds-1)  # kelly formel
    df['spela'] = df.f >0
    df['insats'] = df.spela * df.f * kassa

    df.sort_values(['datum','avd','proba'],ascending=[True,True,False],inplace=True)
    proba_order=df.groupby(['datum','avd']).proba.cumcount()

    df['prob_order']=proba_order+1
    df['y'] = y
    
    print('log(proba)',np.log(df.loc[df.y==1].proba).mean())
    return df, df.loc[df.y==1].prob_order.mean()   # mean prob_order för vinnarhäst

In [3]:
def create_ekipage(df_):
    df=df_.copy()
    prefs = ['','h1_','h2_','h3_','h4_','h5_',]
    for pr in prefs:
        df[pr+'ekipage'] = df[pr+'kusk'].str.cat(df['häst'], sep =", ")
        df.drop([pr+'kusk'],axis=1, inplace=True)
        
    return df.drop(['häst'], axis=1)

In [4]:
### returnera en modell med parametrar satta
def get_model(d=6,l2=2,iterations=3000,use_best=True,verbose=False):
    model = CatBoostClassifier(iterations=iterations,use_best_model=use_best, 
        custom_metric=['Logloss', 'AUC','Recall', 'Precision', 'F1', 'Accuracy'],

        eval_metric='Accuracy', 
        depth=d,l2_leaf_reg=l2,
        auto_class_weights='Balanced',verbose=verbose, random_state=2021) 
    return model                

In [5]:
### Features som inte används vid träning
def remove_features(df_,remove_mer=[]):
    # df = df_.copy()
    #remove_mer=['h5_perf','h5_auto','h4_perf','h4_auto', 'h3_perf', 'h2_perf']
    df = df_.drop(['avd','startnr','vodds','podds','bins','h1_dat','h2_dat','h3_dat','h4_dat','h5_dat'],axis=1) #
    if remove_mer:
        df = df.drop(remove_mer,axis=1)
    
    # df=check_unique(df.copy())
    # df=check_corr(df.copy())
    return df

In [6]:
 ## byt ut alla NaN till text för cat_features
def replace_NaN(X_train,X_test=None, cat_features=[]):
    # print('cat_features',cat_features)
    for c in cat_features:
        # print(c)
        X_train.loc[X_train[c].isna(),c] = 'None'       ### byt ut None-värden till texten 'None
        if X_test is not None:  ## om X_test är med
            X_test.loc [X_test[c].isna(),c] = 'None'    ### byt ut None-värden till texten 'None

    return X_train,X_test

In [7]:
def scrape_nya_lopp():
    nya_lopp,strukna = vs.v75_scraping(resultat=True,history=True)

    df=pd.concat([pd.read_csv('all_data.csv'), nya_lopp])
    print('shape med nya lopp',df.shape)
    #ta bort dubletter
    df.drop_duplicates(['datum','avd','häst'],inplace=True)
    df.sort_values(by=['datum','avd'],inplace=True)
    df.reset_index(drop=True,inplace=True)
    print('shape med dubletter bort',df.shape)

    df.to_csv('all_data.csv', index=False)

    print("första datum i df =",df.datum.head(1).to_list()[0])
    print("sista  datum i df =",df.datum.tail(1).to_list()[0])

    return df,nya_lopp

In [8]:
### beräkna vilka datum att använda ###
def get_alla_datum(test_from_proc=0.75, train_from_proc=0, total_omlärning = False):
    if total_omlärning:
        nya_lopp=None
        df = pd.read_csv('all_data.csv')     
        alla_datum = df.datum.unique()
        split_ix = int(len(alla_datum)*test_from_proc)
    else:
        # normalt adderar vi bara 1 eller flera veckor från "omg_att_spela_link.csv"
        df, nya_lopp = scrape_nya_lopp()  # scrape från 'omg_att_spela_link.csv' och addera till df
        omg_df = pd.read_csv('omg_att_spela_link.csv')     
        startix=omg_df.Link.str.find('spel')[0]    # index till 'spel' i url
        alla_datum = omg_df.Link.str.slice(start=startix+5,stop=startix+15).to_list() # en datum 
        split_ix=0
        print(f'datum att lära från {alla_datum}')

    return df,nya_lopp,alla_datum,split_ix


## Walkthrough-funktionen  här

In [9]:

### Kör en walkthrough learn här, en datum i taget framåt

# Jag har ändrat till att alla steg kör utan test-datam ed fast iterations=100
def walkthrough(classic_test=False, verbose=False):
    
    df, nya_lopp, alla_datum, split_ix = get_alla_datum()

    l2_leaf_regs=2
    model=get_model(use_best=False,iterations=100)
    df=remove_features(df.copy())
    cat_features = list(df.loc[:,df.dtypes=='O'].columns)
    df,_ = replace_NaN(df.copy(), cat_features=cat_features)    
    print(f'cat_features {cat_features}\n')

    df['plac']=(df.plac==1)*1
        
    for nr,datum in enumerate(alla_datum[split_ix:]):
        print(f'walk-iter {nr+1} av {len(alla_datum[split_ix:])} ',end=': ')

        X_train = df.loc[df.datum<datum,:].copy()
        y_train = X_train.plac; X_train.drop(['plac'],axis=1,inplace=True)

        if classic_test:    ### klassisk train/test utan walkthrough
            X_test  = df.loc[df.datum>=datum,:].copy()
            y_test  = X_test.plac;  X_test.drop(['plac'],axis=1,inplace=True)
            train_pool = Pool(X_train,y_train,cat_features=cat_features)
            test_pool = Pool(X_test,y_test,cat_features=cat_features)
            model.fit(train_pool,use_best_model=True, verbose=verbose,eval_set=test_pool)
        else:
            X_test  = df.loc[df.datum==datum,:].copy()
            y_test  = X_test.plac;  X_test.drop(['plac'],axis=1,inplace=True)
            train_pool = Pool(X_train,y_train,cat_features=cat_features)
            test_pool = Pool(X_test,y_test,cat_features=cat_features)
            model.fit(train_pool,use_best_model=False, verbose=verbose)

        print('best iteration',model.get_best_iteration(), '\tbest score', round(model.get_best_score()['learn']['Accuracy'],3) )
        ##['validation']['Logloss'],3),'\t', round(model.get_best_score()['validation']['Accuracy:use_weights=true'],3))
        
        if classic_test:    ### klassisk train/test utan walkthrough
            return model,cat_features
    
        model.save_model('modeller/model_'+datum)

    X_train =df.copy().drop('plac',axis=1)
    y_train = df.plac 
    model.fit(X_train,y=y_train,cat_features=cat_features)
    print(f'spara model_senaste',datum)
    model.save_model('modeller/model_senaste')

    return df,nya_lopp, model,cat_features

### Här körs hela walkthrough

In [10]:
df, nya_lopp, model, cat_features = walkthrough(classic_test=False, verbose=False)

omgång 1: https://www.atg.se/spel/2021-10-23/V75/
klickade på ANPASSA
anpassa klar - break
ant resultat 7
ant lopp 7


2021-10-23 20:35:30.442 WARNING root: 2021-10-23 avd: <selenium.webdriver.remote.webelement.WebElement (session="f3a1c63175922c50523ccc8e04237443", element="a6cb8b6e-c10d-4709-9d67-a61736d11fe9")>


Ant priser 7
ant names,vodds,podds,rader,streck 12 12 12 12


2021-10-23 20:35:30.841 WARNING root: 0


AVD 1 ÖREBRO 2100 AUTOSTART 

...

.

2021-10-23 20:37:26.433 WARNING root: 10


.

2021-10-23 20:37:28.475 WARNING root: 11


..

det tog 120.423 sekunder
utdelning: 5268, 59, 0
startar Fixa mer
tog bort 7 strukna från 87 till 80
rensade totalt bort 7 hästar i städa_och_rensa. Från 87 till 80
shape med nya lopp (42244, 79)
shape med dubletter bort (42244, 79)
första datum i df = 2014-12-28
sista  datum i df = 2021-10-23
datum att lära från ['2021-10-23']
cat_features ['datum', 'bana', 'häst', 'kusk', 'kön', 'h1_kusk', 'h1_bana', 'h2_kusk', 'h2_bana', 'h3_kusk', 'h3_bana', 'h4_kusk', 'h4_bana', 'h5_kusk', 'h5_bana']

walk-iter 1 av 1 : best iteration None 	best score 0.752
spara model_senaste 2021-10-23


## Kör allt ovanför walkthrough
### Se till att "omg_att_spela_link.csv" är ifylld

# Kör cross validation

## init  - kör först allt t.o.m 'replace_NaN()' ovan

In [11]:
model = get_model().load_model('modeller/model_senaste')
dforg = pd.read_csv('all_data.csv')     
# print(df.columns)
df=remove_features(dforg.copy())
# df['avd']=dforg.avd
cat_features = list(df.loc[:,df.dtypes=='O'].columns)
df,_ = replace_NaN(df.copy(), cat_features=cat_features)    
y=df.plac
y=(y==1)*1
df.drop('plac',axis=1,inplace=True)

## cv

In [12]:

cv_pool = Pool(df,y,cat_features=cat_features)

params = {
         'use_best_model': True,
         'eval_metric' : 'AUC',
         "loss_function": "Logloss",
         'early_stopping_rounds': 100,
         'verbose': 50,
}

cv_score =cv(pool=cv_pool, 
   params=params, 
   dtrain=None, 
   iterations=2000, 
   num_boost_round=None,
   fold_count=5, 
   nfold=None,
   inverted=False,
   partition_random_seed=0,
   seed=2021, 
   shuffle=False, 
   logging_level=None, 
   stratified=True,
   as_pandas=True,
   type='TimeSeries')

0:	test: 0.6915885	best: 0.6915885 (0)
50:	test: 0.8119068	best: 0.8119068 (50)
100:	test: 0.8122124	best: 0.8136547 (79)	total: 40.5s	remaining: 12m 41s
150:	test: 0.8112590	best: 0.8136547 (79)
Stopped by overfitting detector  (100 iterations wait)


In [13]:
cv_score

,iterations,test-AUC-mean,test-AUC-std,test-Logloss-mean,test-Logloss-std,train-Logloss-mean,train-Logloss-std
0,0,0.691588,0.048110,0.658933,0.002035,0.658836,0.002075
1,1,0.740145,0.028471,0.626931,0.002848,0.626726,0.002966
2,2,0.746745,0.026317,0.599150,0.002135,0.598843,0.002325
3,3,0.747969,0.027139,0.574382,0.002149,0.574129,0.002351
4,4,0.768854,0.032144,0.549269,0.002992,0.548897,0.003481
...,...,...,...,...,...,...,...
175,175,0.811300,0.005688,0.241053,0.002491,0.208028,0.006911
176,176,0.811226,0.005622,0.241107,0.002449,0.207957,0.006920
177,177,0.811215,0.005622,0.241118,0.002444,0.207885,0.006945
178,178,0.811122,0.005590,0.241150,0.002452,0.207803,0.006913


In [14]:
from IPython.display import display
print(df.datum.max())
display(cv_score[cv_score['test-Logloss-mean'].min() == cv_score['test-Logloss-mean']])
display(cv_score[cv_score['test-AUC-mean'].max() == cv_score['test-AUC-mean']])

2021-10-23


,iterations,test-AUC-mean,test-AUC-std,test-Logloss-mean,test-Logloss-std,train-Logloss-mean,train-Logloss-std
134,134,0.81099,0.005575,0.240641,0.00278,0.212987,0.004837


,iterations,test-AUC-mean,test-AUC-std,test-Logloss-mean,test-Logloss-std,train-Logloss-mean,train-Logloss-std
79,79,0.813655,0.005812,0.244091,0.005533,0.224764,0.002284


In [15]:
from sklearn.model_selection import train_test_split

df[['datum','avd','streck','häst','kusk']] = dforg[['datum','avd','streck','häst','kusk']]

# df.drop('datum',axis=1,inplace=True)
df.drop('avd',axis=1,inplace=True)
df.drop(['streck'],axis=1,inplace=True)
# df.drop(['häst','kusk'],axis=1,inplace=True)
cat_features = list(df.loc[:,df.dtypes=='O'].columns)
X_train,X_test,y_train,y_test = train_test_split(df,y,shuffle=False,)
cb=get_model(use_best=True)
cb.fit(X_train,y_train,eval_set= (X_test,y_test),early_stopping_rounds=200, cat_features=cat_features,verbose=100)

0:	learn: 0.6293511	test: 0.6171433	best: 0.6171433 (0)	total: 104ms	remaining: 5m 13s
100:	learn: 0.7096813	test: 0.6358329	best: 0.6379583 (97)	total: 10.1s	remaining: 4m 48s
200:	learn: 0.7342330	test: 0.6458504	best: 0.6470100 (144)	total: 20.3s	remaining: 4m 42s
300:	learn: 0.7515615	test: 0.6409157	best: 0.6483474 (246)	total: 30.1s	remaining: 4m 29s
400:	learn: 0.7740946	test: 0.6336473	best: 0.6483474 (246)	total: 39.9s	remaining: 4m 18s
Stopped by overfitting detector  (200 iterations wait)

bestTest = 0.6483473906
bestIteration = 246

Shrink model to first 247 iterations.


In [16]:
X_test[['datum','avd','vodds']] = dforg[['datum','avd','vodds']]
predict_prob = cb.predict_proba(X_test)

_,prob_score = proba_order_score(X_test ,y_test, predict_prob)

print('cb med ekipage',prob_score, cb.best_score_['validation']['AUC'])

log(proba) -0.7326576449949074
cb med ekipage 4.079558011049723 0.7021766445275488


C:\Users\peter\anaconda3\lib\site-packages\pandas\core\frame.py:3636: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


# FLAML (med och utan ekipage och streck)

In [17]:
def split_data(df_,train_from_proc=0,test_proc=0.25):
    # train_from_proc = where to start both train and test
    # test_proc = how much of the data is test
    df=df_.copy()
    alla_datum = df.datum.unique()
    train_from_datum = alla_datum[ int(len(alla_datum)*train_from_proc)]
    print(train_from_datum)
    X_test=None
    y_test=None
    test_from_datum=alla_datum[-1]
    if test_proc:
        selected_data = alla_datum[ alla_datum >= train_from_datum ]
        test_from_datum = selected_data[ int(len(selected_data)*(1-test_proc)) ]
        print(test_from_datum)
        X_test  = df[df.datum >= test_from_datum]
        y_test  = (X_test.plac==1)*1
        X_test  = X_test.drop('plac',axis=1)
        
    
    X_train = df[(df.datum >= train_from_datum) & (df.datum < test_from_datum) ]
    y_train = (X_train.plac==1)*1
    
    return X_train.drop('plac', axis=1), X_test, y_train, y_test


In [18]:
# prepare all data för flaml
dforg = pd.read_csv('all_data.csv')  

X_train, X_test, y_train, y_test= split_data(dforg,train_from_proc=0.3,test_proc=0.25)
X_train = remove_features(X_train)
X_test  = remove_features( X_test)

# X_train = X_train.drop('streck', axis=1)
# X_test  = X_test.drop ('streck', axis=1)
# X_train.drop('datum', axis=1, inplace=True)
# X_test.drop( 'datum', axis=1, inplace=True)
cat_features = list(X_train.select_dtypes('object').columns)
# X_train, X_test = replace_NaN(X_train.copy(),X_test=X_test.copy(), cat_features=cat_features) 
# X_train.fillna(-1)
# X_test.fillna(-1)
print(cat_features)
X_train.shape,X_test.shape

2017-02-11
2020-05-30
['datum', 'bana', 'häst', 'kusk', 'kön', 'h1_kusk', 'h1_bana', 'h2_kusk', 'h2_bana', 'h3_kusk', 'h3_bana', 'h4_kusk', 'h4_bana', 'h5_kusk', 'h5_bana']


((22148, 68), (7404, 68))

In [19]:
from flaml import AutoML 
cat_features = list(X_train.select_dtypes('object').columns)
starting_points={'lgbm': {'n_estimators': 38,
  'num_leaves': 4,
  'min_child_samples': 2,
  'learning_rate': 0.19098448074739216,
  'log_max_bin': 7,
  'colsample_bytree': 0.8827412174089042,
  'reg_alpha': 0.004577823970660193,
  'reg_lambda': 0.03815584533462228},
 'rf': {'n_estimators': 33,
  'max_features': 0.3251674877768946,
  'max_leaves': 89,
  'criterion': 'entropy'},
 'catboost': {'early_stopping_rounds': 10,
  'learning_rate': 0.007511731949060241},
 'xgboost': {'n_estimators': 575,
  'max_leaves': 46,
  'min_child_weight': 1.032235057697502,
  'learning_rate': 0.013318439439138472,
  'subsample': 0.7908401179782586,
  'colsample_bylevel': 0.6924750037579576,
  'colsample_bytree': 0.7174828796230647,
  'reg_alpha': 0.15461500385937774,
  'reg_lambda': 0.6619886587472544},
 'extra_tree': {'n_estimators': 47,
  'max_features': 0.7934349565988307,
  'max_leaves': 213,
  'criterion': 'entropy'}}
flml_raw_parms={'task': 'classification','split_type':'time', 'metric':'roc_auc', 'starting_points': starting_points,'verbose':False,
        'time_budget':700, 'max_iter':50000000,'n_jobs':5, 'X_val': X_test, 'y_val':y_test,'early_stop':True, 'ensemble':True}

automl_raw = AutoML()
automl_raw.fit(X_train,y_train, **flml_raw_parms)

[flaml.automl: 10-23 20:52:55] {1541} WARNING - Time taken to find the best model is 73% of the provided time budget and not all estimators' hyperparameter search converged. Consider increasing the time budget.
2021-10-23 20:52:55.791 WARNING flaml.automl: Time taken to find the best model is 73% of the provided time budget and not all estimators' hyperparameter search converged. Consider increasing the time budget.


In [20]:
flm_raw_train_pred= automl_raw.predict_proba(X_train)
flm_raw_test_pred = automl_raw.predict_proba(X_test)

X_test_raw = X_test.copy()
X_test_raw[['datum','avd','vodds']] = dforg[['datum','avd','vodds']]
_,prob_score = proba_order_score(X_test_raw,y_test, flm_raw_test_pred)

print('timeserie, datum,häst, kusk', prob_score, 1-automl_raw.best_loss)
# X_test.columns

log(proba) -1.85365284829531
timeserie, datum,häst, kusk 3.0926216640502355 0.8232616778395663


timeserie  0.3 0.25, datum, häst, kusk 3.720565149136578  0.7213763318649257 ... 1.9827526807785034 .....   best    
timeserie  0.4 0.25, datum, häst, kusk 3.7362637362637363 0.7214144007762124  
timeserie, 0.2 0.25, datum, häst, kusk 3.760989010989011  0.72561915325073230    
timeserie, 0.1 0.25  datum, häst, kusk 3.8180708180708183 0.726597977829505    
timeserie, 0.5 0.25, datum, häst, kusk 3.936263736263736  0.7216626969090024  
timeserie, 0.3 0.25, datum, häst, kusk streck, NaN 3.0706436420722136  0.8230307821948237   
timeserie, 0.3 0.25, datum, häst, kusk,streck  3.0549450549450547 0.8232840226857013 ... -1.7710182666778564 .......... best   
timeserie, 0.3 0.25, datum, häst, kusk streck, NaN, fillna, 3.0549450549450547 0.8237003593459333   
timeserie, 0.3 0.25, datum, häst, kusk, streck 3.06436420722135   0.8232840226857013       
timeserie, 0.4 0.25, datum, häst, kusk, streck 3.1483516483516483 0.8169106155467452  
timeserie, 0.2 0.25, datum, häst, kusk, streck 3.0824175824175826 0.8220287891340522

## Final FLML model

In [21]:
def run_flaml(X_train, y_train):
    import pickle

    for with_streck in [True, False]:
        if with_streck: 
            X_tr = X_train.copy()    
            filename = 'modeller\\FLAML_model.sav'
        else:
            X_tr = X_train.drop('streck', axis=1).copy()
            filename = 'modeller\\FLAML2_model.sav'
            
        print('with_streck = ',with_streck)    
    
        automl = AutoML()
        flml_parms={'task': 'classification','split_type':'time', 'metric':'roc_auc','starting_points': starting_points, 'verbose':False,
        'time_budget':1700, 'max_iter':400000000,'n_jobs':5, 'early_stop':True, 'ensemble':True}

        automl.fit(X_tr, y_train, **flml_parms)
        print(1-automl.best_loss, 'for streck in columns', 'streck' in X_tr.columns)
        
        # save_model
        print('save in',filename)        
        pickle.dump(automl, open(filename, 'wb')) 

In [22]:
# prepare all data för flaml
dforg = pd.read_csv('all_data.csv')  
X_train, _, y_train, _ = split_data(dforg,train_from_proc=0.3,test_proc=None)
X_train = remove_features(X_train)

run_flaml(X_train, y_train)

2017-02-11
with_streck =  True


[flaml.automl: 10-23 21:25:59] {1541} WARNING - Time taken to find the best model is 74% of the provided time budget and not all estimators' hyperparameter search converged. Consider increasing the time budget.
2021-10-23 21:25:59.455 WARNING flaml.automl: Time taken to find the best model is 74% of the provided time budget and not all estimators' hyperparameter search converged. Consider increasing the time budget.


0.8192287829979011 for streck in columns True
save in modeller\FLAML_model.sav
with_streck =  False


[flaml.automl: 10-23 21:59:03] {1541} WARNING - Time taken to find the best model is 75% of the provided time budget and not all estimators' hyperparameter search converged. Consider increasing the time budget.
2021-10-23 21:59:03.481 WARNING flaml.automl: Time taken to find the best model is 75% of the provided time budget and not all estimators' hyperparameter search converged. Consider increasing the time budget.


0.7232698993248787 for streck in columns False
save in modeller\FLAML2_model.sav
